In [1]:
import nltk , string, gensim
from nltk.corpus import abc
from nltk.tokenize import word_tokenize
import numpy as np

In [2]:
import json
jason_file = 'intents.json'
with open('intents.json','r') as f:
    data = json.load(f)

In [3]:
data   

[{'tag': 'welcome',
  'patterns': ['Hi',
   'How are you',
   'Is any one to talk?',
   'Hello',
   'hi are you available'],
  'responses': ['Hello, thanks for contacting us',
   'Good to see you here',
   ' Hi there, how may I assist you?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye', 'I will come back soon'],
  'responses': ['See you later, thanks for visiting',
   'have a great day ahead',
   'Wish you Come back again soon.']},
 {'tag': 'thankful',
  'patterns': ['Thanks for helping me',
   'Thank your guidance',
   "That's helpful and kind from you"],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   'It is my duty to help you']},
 {'tag': 'hoursopening',
  'patterns': ['What hours are you open?',
   'Tell your opening time?',
   'When are you open?',
   'Just your timing please'],
  'responses': ["We're open every day 8am-7pm",
   'Our office hours are 8am-7pm every day',
   'We open office at 8 am and close at 7 pm']},
 {'tag': 'pay

In [4]:
import pandas as pd
df = pd.DataFrame(data)

In [5]:
df

,tag,patterns,responses
0,welcome,"[Hi, How are you, Is any one to talk?, Hello, ...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"[Bye, See you later, Goodbye, I will come back...","[See you later, thanks for visiting, have a gr..."
2,thankful,"[Thanks for helping me, Thank your guidance, T...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"[What hours are you open?, Tell your opening t...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"[Can I pay using credit card?, Can I pay usin...","[We accept VISA, Mastercard and credit card, W..."


In [6]:
df['patterns'] = df['patterns'].apply(", ".join)

In [7]:
df

,tag,patterns,responses
0,welcome,"Hi, How are you, Is any one to talk?, Hello, h...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"Bye, See you later, Goodbye, I will come back ...","[See you later, thanks for visiting, have a gr..."
2,thankful,"Thanks for helping me, Thank your guidance, Th...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"What hours are you open?, Tell your opening ti...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"Can I pay using credit card?, Can I pay using...","[We accept VISA, Mastercard and credit card, W..."


In [8]:
#!pip install textblob
from nltk.corpus import stopwords
from textblob import Word
nltk.download('wordnet')

stop =stopwords.words('english')
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x.lower() for x in word_tokenize(x) if x not in string.punctuation))

df['patterns'] = df['patterns'].str.replace('[^\w\s]','')

df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x.lower() for x in x.split() if not x.isdigit()))

df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x.lower() for x in x.split() if not x in stop))

df['patterns'] = df['patterns'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Error loading wordnet: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>
/tmp/ipykernel_27421/2430501326.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['patterns'] = df['patterns'].str.replace('[^\w\s]','')


In [9]:
from gensim.models import Word2Vec
from nltk.corpus import wordnet

In [10]:
#taking the outer list
bigger_list = []
for i in df['patterns']:
  li = i.split()
  bigger_list.append(li)

In [11]:
bigger_list

[['hi', 'one', 'talk', 'hello', 'hi', 'available'],
 ['bye', 'see', 'later', 'goodbye', 'come', 'back', 'soon'],
 ['thanks', 'helping', 'thank', 'guidance', 'helpful', 'kind'],
 ['hour', 'open', 'tell', 'opening', 'time', 'open', 'timing', 'please'],
 ['pay',
  'using',
  'credit',
  'card',
  'pay',
  'using',
  'mastercard',
  'pay',
  'using',
  'cash']]

In [12]:
model = Word2Vec(bigger_list,min_count = 1, workers =4)
print(model)

Word2Vec<vocab=31, vector_size=100, alpha=0.025>


In [13]:
vocab = list(model.wv.key_to_index)       #saved model can be trained on another machine as well

vocab

['using',
 'pay',
 'hi',
 'open',
 'later',
 'soon',
 'back',
 'come',
 'goodbye',
 'bye',
 'see',
 'helping',
 'available',
 'hello',
 'talk',
 'one',
 'thanks',
 'cash',
 'thank',
 'mastercard',
 'helpful',
 'kind',
 'hour',
 'tell',
 'opening',
 'time',
 'timing',
 'please',
 'credit',
 'card',
 'guidance']

In [14]:
text = "guidance system is not working."

In [15]:
# preprocessing:

def similar_mean(text):
    
    token = word_tokenize(text)

    token2 = [word for word in token if word not in string.punctuation]

    token3 = [word for word in token2 if word not in stopwords.words('english')]
    word_in_vocab = []
    lst = []
    for word in token3:
        if word not in vocab:

            for word in wordnet.synsets(word):
                word1 = word.lemmas()[0].name()
                if word1 not in lst:
                    lst.append(word1)
            print(lst)
        else:
            
            word_in_vocab.append(word)
            similar_word = model.wv.most_similar(word)
            print(similar_word)
    return lst, word_in_vocab


In [16]:
not_in_vocab, in_vocab = similar_mean(text)

[('helpful', 0.18681879341602325), ('helping', 0.18191713094711304), ('later', 0.17530782520771027), ('soon', 0.1626758724451065), ('using', 0.15174217522144318), ('please', 0.1423748880624771), ('cash', 0.1337394416332245), ('credit', 0.11591789871454239), ('kind', 0.10269659012556076), ('thanks', 0.09045052528381348)]
['system', 'arrangement', 'organization']
['system', 'arrangement', 'organization', 'working', 'work', 'function', 'exercise', 'make', 'bring', 'cultivate', 'influence', 'shape', 'knead', 'exploit', 'solve', 'ferment', 'sour', 'running']


In [17]:
not_in_vocab

['system',
 'arrangement',
 'organization',
 'working',
 'work',
 'function',
 'exercise',
 'make',
 'bring',
 'cultivate',
 'influence',
 'shape',
 'knead',
 'exploit',
 'solve',
 'ferment',
 'sour',
 'running']

In [18]:
in_vocab

['guidance']

In [19]:
df

,tag,patterns,responses
0,welcome,hi one talk hello hi available,"[Hello, thanks for contacting us, Good to see ..."
1,goodbye,bye see later goodbye come back soon,"[See you later, thanks for visiting, have a gr..."
2,thankful,thanks helping thank guidance helpful kind,"[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,hour open tell opening time open timing please,"[We're open every day 8am-7pm, Our office hour..."
4,payments,pay using credit card pay using mastercard pay...,"[We accept VISA, Mastercard and credit card, W..."


In [20]:
if in_vocab:
    
    for word in in_vocab:

        a = 0
        
        response = []
        
        for i in df['patterns']:

            if word in i:
                response.append((np.random.choice(df['responses'].iloc[a])))

            a  = a + 1
        print(response)           
else:
    print("no word matches the pattern")
            
            

['It is my duty to help you']
